In [1]:
from data_provider.data_factory import data_provider
from utils.tools import EarlyStopping, adjust_learning_rate, visual, vali, test
from tqdm import tqdm
from models.PatchTST import PatchTST
from models.GPT4TS import GPT4TS
from models.DLinear import DLinear


import numpy as np
import torch
import torch.nn as nn
from torch import optim

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random

import pandas as pd


#couple resnet
import pandas as pd
from datetime import datetime
import numpy as np
import tensorly.tenalg
import numpy as np
import torch
from torch.nn.init import xavier_normal_
import time
from collections import defaultdict
from torch.optim.lr_scheduler import ExponentialLR
import argparse
import array
from collections import defaultdict 

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import matplotlib.pyplot as plt 

#save the result
import pickle

import tensorly.tenalg
import torch.nn as nn
from sklearn.model_selection import KFold
#check cuda
torch.cuda.is_available()

/home/dan/anaconda3/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/dan/anaconda3/envs/geo/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/dan/anaconda3/envs/geo/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/hom

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [14]:
warnings.filterwarnings('ignore')

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [15]:
parser = argparse.ArgumentParser()

parser.add_argument('--model_id', type=str, default='test')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/')

parser.add_argument('--root_path', type=str, default='/home/dan/NeurIPS2023-One-Fits-All/dataset/Data/')
parser.add_argument('--data_path', type=str, default='df_EPS_CHAR.csv')
parser.add_argument('--data', type=str, default='eps')
parser.add_argument('--features', type=str, default='MS')
parser.add_argument('--freq', type=int, default=0)
parser.add_argument('--target', type=str, default='vol')
parser.add_argument('--embed', type=str, default='timeF')
parser.add_argument('--percent', type=int, default=100)

parser.add_argument('--seq_len', type=int, default=19)
parser.add_argument('--pred_len', type=int, default=11)
parser.add_argument('--label_len', type=int, default=9)

parser.add_argument('--decay_fac', type=float, default=0.75)
parser.add_argument('--learning_rate', type=float, default=0.0001)
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--num_workers', type=int, default=10)
parser.add_argument('--train_epochs', type=int, default=10)
parser.add_argument('--lradj', type=str, default='type1')
parser.add_argument('--patience', type=int, default=3)

parser.add_argument('--gpt_layers', type=int, default=3)
parser.add_argument('--is_gpt', type=int, default=1)
parser.add_argument('--e_layers', type=int, default=3)
parser.add_argument('--d_model', type=int, default=768)
parser.add_argument('--n_heads', type=int, default=2)
parser.add_argument('--d_ff', type=int, default=786)
parser.add_argument('--dropout', type=float, default=0.2)
parser.add_argument('--enc_in', type=int, default=18)
parser.add_argument('--c_out', type=int, default=1)
parser.add_argument('--patch_size', type=int, default=4)
parser.add_argument('--kernel_size', type=int, default=25)

parser.add_argument('--loss_func', type=str, default='mse')
parser.add_argument('--pretrain', type=int, default=1)
parser.add_argument('--freeze', type=int, default=1)
parser.add_argument('--model', type=str, default='model')
parser.add_argument('--stride', type=int, default=1)
parser.add_argument('--max_len', type=int, default=-1)
parser.add_argument('--hid_dim', type=int, default=16)
parser.add_argument('--tmax', type=int, default=10)

parser.add_argument('--itr', type=int, default=1)
parser.add_argument('--cos', type=int, default=0)
parser.add_argument('--which_firm', type=int, default=0)
parser.add_argument('--num_variables', type=int, default=18)

#args = parser.parse_known_args()[0]
args = parser.parse_args(args=[])

In [16]:
    def month_each_firm(df_EPS,idx):
        select_columns=['date','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']
        tickers=df_EPS['ticker'].unique()
        df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})

        df_0=df_EPS[df_EPS['ticker']==tickers[idx]][select_columns]
        final_0 = pd.merge(df_date, df_0, how='left', on='date')
        df_impute=final_0.iloc[:,1:-1].fillna(0)
        df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)
        return(df_raw)


In [23]:
df_EPS = pd.read_csv(os.path.join(args.root_path,
                                          args.data_path))

In [24]:
df_EPS

,ticker,pends,ym,permno,gvkey,datadate,yq,cusip,value,anndats,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,date
0,DANS,NaN,198001,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01
1,DANS,NaN,198002,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-02-01
2,DANS,NaN,198003,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-03-01
3,DANS,NaN,198004,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-04-01
4,DANS,NaN,198005,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236220,023A,2023-07-31,202307,17685.0,33301.0,2023-04-30,202303.0,25616310,0.72,2023-09-07,...,0.009274,-0.003308,-0.044046,51.090000,-0.094149,1313675.0,202371.0,0.66,0.66,2023-07-01
2236221,ANF,2023-07-31,202307,83976.0,63643.0,2023-04-30,202303.0,00289620,1.10,2023-08-23,...,-0.043901,-0.066070,0.113301,37.680000,0.214309,350453.0,50065.0,0.14,0.13,2023-07-01
2236222,ORCL,2023-08-31,202308,10104.0,12142.0,2023-05-31,202303.0,68389X10,1.19,2023-09-11,...,0.089011,-0.007677,-0.067191,117.230003,-0.012260,1563054.0,2714259.0,1.14,1.14,2023-08-01
2236223,ADBE,2023-08-31,202308,75510.0,12540.0,2023-05-31,202303.0,00724F10,4.09,2023-09-14,...,-0.292729,-0.004173,0.019000,546.169983,0.116935,607578.0,455800.0,3.88,3.89,2023-08-01


In [25]:
different_values =df_EPS.groupby('cusip')['ticker'].nunique() > 1

In [26]:
different_values.sum()

16

In [27]:
df_EPS['ticker'].nunique()

12721

In [7]:
data_EPS_list=[]
for i in range(df_EPS['ticker'].nunique()):
    df_raw=month_each_firm(df_EPS,i)
    data_EPS_list.append(torch.Tensor(df_raw.drop('date',axis=1).values))
    if i%100==0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700


In [9]:
data_EPS1=torch.stack(data_EPS_list,axis=1)
data_EPS1.shape

torch.Size([524, 12721, 19])

In [12]:
np.save('/home/dan/NeurIPS2023-One-Fits-All1/Long-term_Forecasting/monthly_eps_tensor.npy',data_EPS1.numpy())

In [16]:
df_raw=quarter_each_firm(df_EPS,0)
df_raw

,date,atq_1,atq_2,atq_3,ni_1,ni_2,ni_3,dv_1,dv_2,dv_3,...,shrout_1,shrout_2,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value
0,1980-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1980-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.04
2,1980-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1980-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1981-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2022-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
170,2022-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
171,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
172,2023-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [12]:
data_EPS=torch.stack(data_EPS_list,axis=1)

In [14]:
df_draw

NameError: name 'df_draw' is not defined

In [13]:
data_EPS.shape

torch.Size([174, 10, 55])

In [21]:
####################
#split the input output sequence
import numpy as np

# Generate a sample numpy array of length 180
np.random.seed(0)  # For reproducibility
data = np.array(range(data_EPS.shape[0]))

# Define lag and future steps
lag = 20
future_steps = 3

# Create rolling window arrays for X (features) and y (target)
X = []
y = []
for i in range(len(data)-lag-future_steps+1):
    X.append(data[i:i+lag])  # Use data from i to i+lag-1
    y.append(data[i+lag:i+lag+future_steps])  # Predict future_steps values ahead

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (152, 20)
y shape: (152, 3)


In [22]:
X[151,:]

array([151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170])

In [23]:
y[-1,:]

array([171, 172, 173])

In [42]:
X_train=X[0:90,:]
y_train=y[0:90,:]
X_val=X[90:110,:]
y_val=y[90:110,:]
X_test=X[110:,:]
y_test=y[110:,:]

In [44]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path)
        self.val_loss_min = val_loss

In [45]:
#all the functions need to use later

def plot_loss(dic):
    plt.figure(figsize=(12,5))
    plt.subplot(1, 2, 1)
    plt.plot(dic['train_loss'], label='Training loss ')
    plt.plot(dic['val_loss'], label='Validation loss ')
    plt.legend(frameon=False)
    plt.title('train and validation loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')

    plt.subplot(1, 2, 2)
    plt.plot(dic['val_mae'])
    plt.title('validation MAE')
    plt.xlabel('epoch')
    plt.ylabel('MAE')
    plt.show()
    
def multi_mode_dot(tensor, matrix_or_vec_list, modes=None, skip=None):
    if modes is None:
        modes = range(len(matrix_or_vec_list))

    decrement = 0  # If we multiply by a vector, we diminish the dimension of the tensor

    res = tensor

    # Order of mode dots doesn't matter for different modes
    # Sorting by mode shouldn't change order for equal modes
    factors_modes = sorted(zip(matrix_or_vec_list, modes), key=lambda x: x[1])
    for i, (matrix_or_vec, mode) in enumerate(factors_modes):
        if (skip is not None) and (i == skip):
            continue

        if mode==2:
            res = tensorly.tenalg.mode_dot(res, matrix_or_vec, mode - decrement)
        else:
            res = tensorly.tenalg.mode_dot(res, matrix_or_vec, mode - decrement)
            res=torch.nn.functional.leaky_relu_(torch.tensor(res)) 
            res=res.detach().numpy()
            
        
        

        if np.ndim(matrix_or_vec) == 1:
            decrement += 1

    return res

def mape(y_true, y_pred, threshold=0.1):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return  100.0 * np.mean(diff, axis=-1).mean()

def mae(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

#use relu or leaky_relu function
def nonlinear(active,vector):
    if active=='relu':
        return torch.nn.functional.relu_(vector)
    elif active == 'leaky_relu':
        return torch.nn.functional.leaky_relu_(vector)
    
#use relu or leaky_relu layer    
def non_layer(active):
    if active=='relu':
        return nn.ReLU()
    elif active=='leaky_relu':
        return nn.LeakyReLU()
    
def train_val_split(tr_idxs,tr_vals,validation_split):
    validation_split = validation_split
    dataset_size = len(tr_vals)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    np.random.seed(1337)
    np.random.shuffle(indices)
    train_indices, valid_indices = indices[split:], indices[:split]
    tr_idx=tr_idxs[train_indices]
    tr_val=tr_vals[train_indices].reshape(-1,1)
    val_idx=tr_idxs[valid_indices]
    val_val=tr_vals[valid_indices].reshape(-1,1)
    trainset=np.concatenate((tr_idx,tr_val),axis=1)
    valset=np.concatenate((val_idx,val_val),axis=1)
    return trainset,valset

def get_result(experiment,model,tr_idxs,tr_vals):
    _,mse,rmse,mae,mape=experiment.evaluate(model,tr_idxs,tr_vals)
    print('mse:',mse,'rmse:',rmse,'mae:',mae,'mape:',mape)

In [46]:
#shi zhe ge
class ResNet_CP(torch.nn.Module):
    def __init__(self,shape,Rank_CP,rank_CP,layer,batch_size,active,Cuda,**kwargs):
        super(ResNet_CP, self).__init__()
        
    
        #original core initial parameters
        self.E = torch.nn.Embedding(shape[0], Rank_CP, padding_idx=0)#.cuda()#common mode XY
        self.R = torch.nn.Embedding(shape[1], Rank_CP, padding_idx=0)#.cuda()#common mode XY
        self.P = torch.nn.Embedding(shape[2], Rank_CP, padding_idx=0)#.cuda()#individual mode for tensor X
        if Cuda:
            print(Cuda)
            self.E = self.E.cuda()
            self.R =self.R.cuda()
            self.P =self.P.cuda()  
 
        self.loss = torch.nn.MSELoss()
        
        self.Rank_CP=Rank_CP
        self.rank_CP=rank_CP
        self.batch_size=batch_size
        self.layer=layer
        self.active=active
        
    def init(self):
        xavier_normal_(self.E.weight.data)
        xavier_normal_(self.R.weight.data)
        xavier_normal_(self.P.weight.data)
        

    def forward(self, e_idx, r_idx, p_idx):
        Esum=self.E.weight
        Rsum=self.R.weight
        Psum=self.P.weight
        
       
        e = Esum[e_idx,:]
            #e = self.bn0(e) #batch
        e = e.view(-1,self.Rank_CP) #b*R
        r = Rsum[r_idx,:]
            #r = self.bn1(r)#batch
        r = r.view(-1,self.Rank_CP) #b*R
        p = Psum[p_idx,:]
            #p = self.bn2(p)#batch
        p = p.view(-1,self.Rank_CP) #b*R
            
        #CP
        reconstruct_tensor=torch.zeros(e.shape[0],device="cuda")
        for i in range(self.Rank_CP):
            reconstruct_tensor+= e[:,i] * r[:,i] * p[:,i]
        pred=reconstruct_tensor
            #print(pred)
            #print(pred.shape)
        return(pred)

In [4]:
SEASONALITY_MAP = {
   "minutely": 1440,
   "10_minutes": 144,
   "half_hourly": 48,
   "hourly": 24,
   "daily": 7,
   "weekly": 1,
   "monthly": 12,
   "quarterly": 4,
   "yearly": 1
}

In [5]:
# Function to convert 'yyyymm' to 'yyyyqq'
def convert_to_quarter(yyyymm):
    year = yyyymm // 100
    month = yyyymm % 100
    if 1 <= month <= 3:
        quarter = 1
    elif 4 <= month <= 6:
        quarter = 2
    elif 7 <= month <= 9:
        quarter = 3
    else:
        quarter = 4
    return f"{year}0{quarter}"

# Define function to flatten columns into a single row for each group
def flatten_columns(group):
    flattened = {}
    for col in group.columns:
        # Create a new column for each unique entry in the original column
        values = group[col].values
        for i, value in enumerate(values):
            new_col_name = f"{col}_{i+1}"
            flattened[new_col_name] = value
    return pd.Series(flattened)


def quarter_each_firm(df_EPS,idx):
    #all the different firms
    tickers=df_EPS['ticker'].unique()
    #the date and input variable, value is the target
    select_columns=['yq','ym','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

    #make sure all the month over different firms is same
    df_date=pd.DataFrame({'ym':df_EPS['ym'].dropna().unique()})
    df_0=df_EPS[df_EPS['ticker']==tickers[idx]][select_columns]
    final_0 = pd.merge(df_date, df_0, how='left', on='ym')

    ###########################################
    #deal with the 'yq' has missing information
    #remove the column 'yq' cause have missing value
    final_0 = final_0.drop('yq', axis=1)
    #Generate new yq column based on ym
    # Apply the conversion function to the column
    final_0['yq'] = final_0['ym'].apply(convert_to_quarter)

    #remove the quarter with less than 3 months
    final_0 = final_0[final_0['yq'] != '202303']

    #impute the input value with missing by 0
    df_impute=final_0.fillna(0)
    df_raw = pd.concat([final_0[['yq','ym']], df_impute[select_columns[2:-1]], final_0['value']], axis=1)


    # Apply the function to each group after grouping by 'yq'
    result_df = df_raw.groupby('yq').apply(flatten_columns).reset_index()

    #remove some columns
    columns_to_remove = ['yq', 'yq_1', 'yq_2', 'yq_3', 'ym_1', 'ym_2','value_1', 'value_2']
    # Drop the specified columns
    result_df = result_df.drop(columns=columns_to_remove)

    #convert string format qarter to date type (using the last month of each quarter)
    result_df['ym_3'] = pd.to_datetime(result_df['ym_3'].astype(str),format='%Y%m')
    result_df = result_df.rename(columns={'ym_3': 'date', 'value_3': 'value'})
    return(result_df)

In [6]:
df_EPS = pd.read_csv(os.path.join(args.root_path,args.data_path))
df_EPS

,ticker,pends,ym,permno,gvkey,datadate,yq,cusip,value,anndats,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,date
0,DANS,NaN,198001,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01
1,DANS,NaN,198002,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-02-01
2,DANS,NaN,198003,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-03-01
3,DANS,NaN,198004,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-04-01
4,DANS,NaN,198005,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236220,023A,2023-07-31,202307,17685.0,33301.0,2023-04-30,202303.0,25616310,0.72,2023-09-07,...,0.009274,-0.003308,-0.044046,51.090000,-0.094149,1313675.0,202371.0,0.66,0.66,2023-07-01
2236221,ANF,2023-07-31,202307,83976.0,63643.0,2023-04-30,202303.0,00289620,1.10,2023-08-23,...,-0.043901,-0.066070,0.113301,37.680000,0.214309,350453.0,50065.0,0.14,0.13,2023-07-01
2236222,ORCL,2023-08-31,202308,10104.0,12142.0,2023-05-31,202303.0,68389X10,1.19,2023-09-11,...,0.089011,-0.007677,-0.067191,117.230003,-0.012260,1563054.0,2714259.0,1.14,1.14,2023-08-01
2236223,ADBE,2023-08-31,202308,75510.0,12540.0,2023-05-31,202303.0,00724F10,4.09,2023-09-14,...,-0.292729,-0.004173,0.019000,546.169983,0.116935,607578.0,455800.0,3.88,3.89,2023-08-01


In [74]:
obs_firm=[]
for i in range(df_EPS['ticker'].nunique()):
    df=quarter_each_firm(df_EPS,i)
    tr_num=(~df.iloc[:92]['value'].isnull()).sum()
    vali_num=(~df.iloc[92:92+18]['value'].isnull()).sum()
    te_num=(~df.iloc[92+18:]['value'].isnull()).sum()
    if tr_num>=5 and vali_num>=5 and te_num>=5:
        print(f'firm: {i}, train count: {tr_num}, validation count: {vali_num}, test count: {te_num}')
        obs_firm.append(i)

firm: 2, train count: 30, validation count: 12, test count: 39
firm: 3, train count: 81, validation count: 18, test count: 15
firm: 13, train count: 50, validation count: 12, test count: 48
firm: 18, train count: 45, validation count: 10, test count: 9
firm: 19, train count: 41, validation count: 14, test count: 9
firm: 21, train count: 81, validation count: 18, test count: 64
firm: 22, train count: 45, validation count: 18, test count: 54
firm: 23, train count: 77, validation count: 18, test count: 64
firm: 29, train count: 42, validation count: 18, test count: 18
firm: 34, train count: 76, validation count: 18, test count: 64
firm: 35, train count: 57, validation count: 14, test count: 8
firm: 38, train count: 80, validation count: 18, test count: 64
firm: 40, train count: 72, validation count: 18, test count: 53
firm: 46, train count: 74, validation count: 17, test count: 9
firm: 47, train count: 69, validation count: 17, test count: 5
firm: 49, train count: 81, validation count: 18

In [8]:
df

,date,atq_1,atq_2,atq_3,ni_1,ni_2,ni_3,dv_1,dv_2,dv_3,...,shrout_1,shrout_2,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value
0,1980-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1980-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.04
2,1980-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1980-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1981-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2022-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
170,2022-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
171,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
172,2023-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [65]:
df=quarter_each_firm(df_EPS,2)

In [66]:
(~df.iloc[:92]['value'].isnull()).sum()

30

In [67]:
(~df.iloc[92:92+18]['value'].isnull()).sum()

12

In [68]:
(~df.iloc[92+18:]['value'].isnull()).sum()

39

In [6]:
2,3,13,18,19,21,22,23,29,34,35,38,40,46,47,49,51,58,60,63,64,66,79,83,85,86,88,89,93,96,101,102,117,119

,ym,yq,atq,ni,dv,acc,invest,mc,bm,dinvt,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,198001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,198002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,198003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,198005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,202304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520,202305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
521,202306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,202307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:

final_0

,ym,atq,ni,dv,acc,invest,mc,bm,dinvt,dar,...,gm,sga,prc,ret,vol,shrout,medest,meanest,value,yq
0,198001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198001
1,198002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198001
2,198003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198001
3,198004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198002
4,198005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,202302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202301
518,202303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202301
519,202304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202302
520,202305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202302


In [25]:
final_0

,ym,atq,ni,dv,acc,invest,mc,bm,dinvt,dar,...,gm,sga,prc,ret,vol,shrout,medest,meanest,value,yq
0,198001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198001
1,198002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198001
2,198003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198001
3,198004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198002
4,198005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,202302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202301
518,202303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202301
519,202304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202302
520,202305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202302


In [28]:

df_raw

,yq,ym,atq,ni,dv,acc,invest,mc,bm,dinvt,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,198001,198001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,198001,198002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,198001,198003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,198002,198004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,198002,198005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,202301,202302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
518,202301,202303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519,202302,202304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
520,202302,202305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [29]:


# Display the resulting DataFrame
result_df

,yq,yq_1,yq_2,yq_3,ym_1,ym_2,ym_3,atq_1,atq_2,atq_3,...,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value_1,value_2,value_3
0,198001,198001,198001,198001,198001,198002,198003,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,198002,198002,198002,198002,198004,198005,198006,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-0.04
2,198003,198003,198003,198003,198007,198008,198009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,198004,198004,198004,198004,198010,198011,198012,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,198101,198101,198101,198101,198101,198102,198103,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,202202,202202,202202,202202,202204,202205,202206,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
170,202203,202203,202203,202203,202207,202208,202209,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
171,202204,202204,202204,202204,202210,202211,202212,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
172,202301,202301,202301,202301,202301,202302,202303,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [30]:
result_df.columns

Index(['yq', 'yq_1', 'yq_2', 'yq_3', 'ym_1', 'ym_2', 'ym_3', 'atq_1', 'atq_2',
       'atq_3', 'ni_1', 'ni_2', 'ni_3', 'dv_1', 'dv_2', 'dv_3', 'acc_1',
       'acc_2', 'acc_3', 'invest_1', 'invest_2', 'invest_3', 'mc_1', 'mc_2',
       'mc_3', 'bm_1', 'bm_2', 'bm_3', 'dinvt_1', 'dinvt_2', 'dinvt_3',
       'dar_1', 'dar_2', 'dar_3', 'capx_1', 'capx_2', 'capx_3', 'gm_1', 'gm_2',
       'gm_3', 'sga_1', 'sga_2', 'sga_3', 'prc_1', 'prc_2', 'prc_3', 'ret_1',
       'ret_2', 'ret_3', 'vol_1', 'vol_2', 'vol_3', 'shrout_1', 'shrout_2',
       'shrout_3', 'medest_1', 'medest_2', 'medest_3', 'meanest_1',
       'meanest_2', 'meanest_3', 'value_1', 'value_2', 'value_3'],
      dtype='object')

In [32]:
result_df

,ym_3,atq_1,atq_2,atq_3,ni_1,ni_2,ni_3,dv_1,dv_2,dv_3,...,shrout_1,shrout_2,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value_3
0,198003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,198006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.04
2,198009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,198012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,198103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,202206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
170,202209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
171,202212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
172,202303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [36]:
result_df

,date,atq_1,atq_2,atq_3,ni_1,ni_2,ni_3,dv_1,dv_2,dv_3,...,shrout_1,shrout_2,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value
0,1980-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1980-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.04
2,1980-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1980-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1981-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2022-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
170,2022-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
171,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
172,2023-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
import pandas as pd

# Sample data
data = {'yq': ['199301', '199302', '199303', '199304']}
df_EPS = pd.DataFrame(data)

# Function to convert 'yyyyqq' to datetime
def yq_to_datetime(yq_str):
    year = int(yq_str[:4])
    quarter = int(yq_str[4:6])
    month = (quarter - 1) * 3 + 1  # Map 1->1, 2->4, 3->7, 4->10
    return pd.Timestamp(f'{year}-{month:02d}-01')

# Apply the conversion function to the DataFrame
df_EPS['date'] = df_EPS['yq'].apply(yq_to_datetime)

print(df_EPS)

In [5]:
df_EPS = pd.read_csv(os.path.join(args.root_path,args.data_path))
df_EPS

,ticker,pends,ym,permno,gvkey,datadate,yq,cusip,value,anndats,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,date
0,DANS,NaN,198001,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01
1,DANS,NaN,198002,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-02-01
2,DANS,NaN,198003,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-03-01
3,DANS,NaN,198004,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-04-01
4,DANS,NaN,198005,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236220,023A,2023-07-31,202307,17685.0,33301.0,2023-04-30,202303.0,25616310,0.72,2023-09-07,...,0.009274,-0.003308,-0.044046,51.090000,-0.094149,1313675.0,202371.0,0.66,0.66,2023-07-01
2236221,ANF,2023-07-31,202307,83976.0,63643.0,2023-04-30,202303.0,00289620,1.10,2023-08-23,...,-0.043901,-0.066070,0.113301,37.680000,0.214309,350453.0,50065.0,0.14,0.13,2023-07-01
2236222,ORCL,2023-08-31,202308,10104.0,12142.0,2023-05-31,202303.0,68389X10,1.19,2023-09-11,...,0.089011,-0.007677,-0.067191,117.230003,-0.012260,1563054.0,2714259.0,1.14,1.14,2023-08-01
2236223,ADBE,2023-08-31,202308,75510.0,12540.0,2023-05-31,202303.0,00724F10,4.09,2023-09-14,...,-0.292729,-0.004173,0.019000,546.169983,0.116935,607578.0,455800.0,3.88,3.89,2023-08-01


In [13]:
#remove the column 'yq' cause have missing value
df_EPS = df_EPS.drop('yq', axis=1)

In [18]:
#Generate new yq column based on ym
# Function to convert 'yyyymm' to 'yyyyqq'
def convert_to_quarter(yyyymm):
    year = yyyymm // 100
    month = yyyymm % 100
    if 1 <= month <= 3:
        quarter = 1
    elif 4 <= month <= 6:
        quarter = 2
    elif 7 <= month <= 9:
        quarter = 3
    else:
        quarter = 4
    return f"{year}0{quarter}"

# Apply the conversion function to the column
df_EPS['yq'] = df_EPS['ym'].apply(convert_to_quarter)

In [21]:
tickers=df_EPS['ticker'].unique()
select_columns=['yq','ym','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

#df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})
df_0=df_EPS[df_EPS['ticker']==tickers[0]][select_columns]
#final_0 = pd.merge(df_date, df_0, how='left', on='date')
#df_impute=final_0.iloc[:,1:-1].fillna(0)
#df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)

In [22]:
df_0

,yq,ym,atq,ni,dv,acc,invest,mc,bm,dinvt,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,198001,198001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,198001,198002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,198001,198003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198002,198004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,198002,198005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099281,200201,200202,41.0,-4.574,NaN,-0.456,-7.092,1.237606,NaN,0.074433,...,0.391155,0.117198,0.128440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1107318,200201,200203,41.0,-4.574,NaN,-0.456,-7.092,1.237606,NaN,0.074433,...,0.391155,0.117198,0.128440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1111738,200202,200204,42.9,-0.228,0.0,1.326,1.900,1.419927,NaN,-0.203119,...,0.091038,-0.283581,-0.161451,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117372,200202,200205,42.9,-0.228,0.0,1.326,1.900,1.419927,NaN,-0.203119,...,0.091038,-0.283581,-0.161451,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:


# Define function to flatten columns into a single row for each group
def flatten_columns(group):
    flattened = {}
    for col in group.columns:
        # Create a new column for each unique entry in the original column
        values = group[col].values
        for i, value in enumerate(values):
            new_col_name = f"{col}_{i+1}"
            flattened[new_col_name] = value
    return pd.Series(flattened)

# Apply the function to each group after grouping by 'yq'
result_df = df_0.groupby('yq').apply(flatten_columns).reset_index()

# Display the resulting DataFrame
result_df

,yq,yq_1,yq_2,yq_3,ym_1,ym_2,ym_3,atq_1,atq_2,atq_3,...,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value_1,value_2,value_3
0,198001,198001,198001,198001,198001,198002,198003,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,198002,198002,198002,198002,198004,198005,198006,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04
2,198003,198003,198003,198003,198007,198008,198009,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198004,198004,198004,198004,198010,198011,198012,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,198101,198101,198101,198101,198101,198102,198103,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,200102,200102,200102,200102,200104,200105,200106,50.238,50.238,50.238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,200103,200103,200103,200103,200107,200108,200109,48.087,48.087,48.087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,200104,200104,200104,200104,200110,200111,200112,48.092,48.092,48.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,200201,200201,200201,200201,200201,200202,200203,41.000,41.000,41.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:

# Convert the 'yyyyqq' column to integers
result_df['yq'] = pd.to_numeric(result_df['yq'], errors='coerce')

# Function to convert 'yyyyqq' to a date
def quarter_to_date(yyyyqq):
    year = yyyyqq // 100  # Extracts the year part
    quarter = yyyyqq % 100  # Extracts the quarter part

    # Map quarters to the first month of the quarter
    quarter_start_month = {1: 1, 2: 4, 3: 7, 4: 10}
    month = quarter_start_month.get(quarter, 1)  # Default to January if the quarter is missing or invalid

    # Return a date representing the first day of the specified quarter
    return pd.Timestamp(year, month, 1)

# Apply the function to convert the 'yyyyqq' column to a date
result_df['date'] = result_df['yq'].apply(quarter_to_date)

# Display the DataFrame with the new date column
result_df

,yq,yq_1,yq_2,yq_3,ym_1,ym_2,ym_3,atq_1,atq_2,atq_3,...,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value_1,value_2,value_3,quarter_start_date,date
0,198001,198001,198001,198001,198001,198002,198003,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01,1980-01-01
1,198002,198002,198002,198002,198004,198005,198006,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04,1980-04-01,1980-04-01
2,198003,198003,198003,198003,198007,198008,198009,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-07-01,1980-07-01
3,198004,198004,198004,198004,198010,198011,198012,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-10-01,1980-10-01
4,198101,198101,198101,198101,198101,198102,198103,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981-01-01,1981-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,200102,200102,200102,200102,200104,200105,200106,50.238,50.238,50.238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-04-01,2001-04-01
86,200103,200103,200103,200103,200107,200108,200109,48.087,48.087,48.087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-07-01,2001-07-01
87,200104,200104,200104,200104,200110,200111,200112,48.092,48.092,48.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-10-01,2001-10-01
88,200201,200201,200201,200201,200201,200202,200203,41.000,41.000,41.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-01-01,2002-01-01


In [49]:
select_columns=['date','atq_1', 'atq_2',
       'atq_3', 'ni_1', 'ni_2', 'ni_3', 'dv_1', 'dv_2', 'dv_3', 'acc_1',
       'acc_2', 'acc_3', 'invest_1', 'invest_2', 'invest_3', 'mc_1', 'mc_2',
       'mc_3', 'bm_1', 'bm_2', 'bm_3', 'dinvt_1', 'dinvt_2', 'dinvt_3',
       'dar_1', 'dar_2', 'dar_3', 'capx_1', 'capx_2', 'capx_3', 'gm_1', 'gm_2',
       'gm_3', 'sga_1', 'sga_2', 'sga_3', 'prc_1', 'prc_2', 'prc_3', 'ret_1',
       'ret_2', 'ret_3', 'vol_1', 'vol_2', 'vol_3', 'shrout_1', 'shrout_2',
       'shrout_3', 'medest_1', 'medest_2', 'medest_3', 'meanest_1',
       'meanest_2', 'meanest_3','value_3']

final_df=result_df[select_columns]
final_df

,date,atq_1,atq_2,atq_3,ni_1,ni_2,ni_3,dv_1,dv_2,dv_3,...,shrout_1,shrout_2,shrout_3,medest_1,medest_2,medest_3,meanest_1,meanest_2,meanest_3,value_3
0,1980-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1980-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04
2,1980-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1981-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2001-04-01,50.238,50.238,50.238,-2.080,-2.080,-2.080,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,2001-07-01,48.087,48.087,48.087,-2.226,-2.226,-2.226,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,2001-10-01,48.092,48.092,48.092,-1.138,-1.138,-1.138,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,2002-01-01,41.000,41.000,41.000,-4.574,-4.574,-4.574,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
#####################
#construct quarterly data for GPT

#load the data
df_EPS = pd.read_csv(os.path.join(args.root_path,args.data_path))

#deal with the 'yq' has missing information
#remove the column 'yq' cause have missing value
df_EPS = df_EPS.drop('yq', axis=1)
#Generate new yq column based on ym
# Function to convert 'yyyymm' to 'yyyyqq'
def convert_to_quarter(yyyymm):
    year = yyyymm // 100
    month = yyyymm % 100
    if 1 <= month <= 3:
        quarter = 1
    elif 4 <= month <= 6:
        quarter = 2
    elif 7 <= month <= 9:
        quarter = 3
    else:
        quarter = 4
    return f"{year}0{quarter}"

# Apply the conversion function to the column
df_EPS['yq'] = df_EPS['ym'].apply(convert_to_quarter)

tickers=df_EPS['ticker'].unique()
select_columns=['yq','ym','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

#make sure all the month over different firms is same
df_date=pd.DataFrame({'ym':df_EPS['ym'].dropna().unique()})
df_0=df_EPS[df_EPS['ticker']==tickers[0]][select_columns]

final_0 = pd.merge(df_date, df_0, how='left', on='ym')
df_impute=final_0.iloc[:,1:-1].fillna(0)
df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)



# Define function to flatten columns into a single row for each group
def flatten_columns(group):
    flattened = {}
    for col in group.columns:
        # Create a new column for each unique entry in the original column
        values = group[col].values
        for i, value in enumerate(values):
            new_col_name = f"{col}_{i+1}"
            flattened[new_col_name] = value
    return pd.Series(flattened)

# Apply the function to each group after grouping by 'yq'
result_df = df_raw.groupby('yq').apply(flatten_columns).reset_index()

# Display the resulting DataFrame
result_df


,yq,level_1,0
0,0,ym_1,200207.0
1,0,ym_2,200208.0
2,0,ym_3,200209.0
3,0,ym_4,200210.0
4,0,ym_5,200211.0
...,...,...,...
10999,200202,meanest_2,0.0
11000,200202,meanest_3,0.0
11001,200202,value_1,NaN
11002,200202,value_2,NaN


In [73]:
#####################
#construct quarterly data for GPT

#load the data
df_EPS = pd.read_csv(os.path.join(args.root_path,args.data_path))
tickers=df_EPS['ticker'].unique()
select_columns=['yq','ym','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

#select one firm
df_0=df_EPS[df_EPS['ticker']==tickers[0]][select_columns]

,yq,yq_1,ym_1,atq_1,ni_1,dv_1,acc_1,invest_1,mc_1,bm_1,...,capx_1,gm_1,sga_1,prc_1,ret_1,vol_1,shrout_1,medest_1,meanest_1,value_1
0,198002.0,198002.0,198006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0400
1,198703.0,198703.0,198709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,198704.0,198704.0,198712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198801.0,198801.0,198803.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,198802.0,198802.0,198806.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,198803.0,198803.0,198809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,198804.0,198804.0,198812.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,198901.0,198901.0,198903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,198902.0,198902.0,198906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,198903.0,198903.0,198909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
final_0

,yq,yq_1,ym_1,atq_1,ni_1,dv_1,acc_1,invest_1,mc_1,bm_1,...,capx_1,gm_1,sga_1,prc_1,ret_1,vol_1,shrout_1,medest_1,meanest_1,value_1
0,198002.0,198002.0,198006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04
1,198102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,198201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,198203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,202203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,202204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,202301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,202302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_raw['yq'].unique()

array(['198001', '198002', '198003', '198004', '198101', '198102',
       '198103', '198104', '198201', '198202', '198203', '198204',
       '198301', '198302', '198303', '198304', '198401', '198402',
       '198403', '198404', '198501', '198502', '198503', '198504',
       '198601', '198602', '198603', '198604', '198701', '198702',
       '198703', '198704', '198801', '198802', '198803', '198804',
       '198901', '198902', '198903', '198904', '199001', '199002',
       '199003', '199004', '199101', '199102', '199103', '199104',
       '199201', '199202', '199203', '199204', '199301', '199302',
       '199303', '199304', '199401', '199402', '199403', '199404',
       '199501', '199502', '199503', '199504', '199601', '199602',
       '199603', '199604', '199701', '199702', '199703', '199704',
       '199801', '199802', '199803', '199804', '199901', '199902',
       '199903', '199904', '200001', '200002', '200003', '200004',
       '200101', '200102', '200103', '200104', '200201', '2002

In [70]:
# Define function to flatten columns into a single row for each group
def flatten_columns(group):
    flattened = {}
    for col in group.columns:
        # Create a new column for each unique entry in the original column
        values = group[col].values
        for i, value in enumerate(values):
            new_col_name = f"{col}_{i+1}"
            flattened[new_col_name] = value
    return pd.Series(flattened)

# Apply the function to each group after grouping by 'yq'
result_df = final_0.groupby('yq').apply(flatten_columns).reset_index()

# Display the resulting DataFrame
result_df

,yq,level_1,0
0,198001,ym_1,198001
1,198001,ym_2,198002
2,198001,ym_3,198003
3,198001,atq_1,NaN
4,198001,atq_2,NaN
...,...,...,...
10999,202303,meanest_2,NaN
11000,202303,value_1,NaN
11001,202303,value_2,NaN
11002,202303,yq_1,202303


In [60]:
final_0 = pd.merge(df_date, df_0, how='left', on='ym')
df_impute=final_0.iloc[:,1:-1].fillna(0)
df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)
df_raw

,ym,yq,atq,ni,dv,acc,invest,mc,bm,dinvt,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,198001,198001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,198002,198001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,198003,198001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,198004,198002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,198005,198002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,202304,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
520,202305,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
521,202306,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
522,202307,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [66]:

final_0 = pd.merge(df_date, df_0, how='left', on='ym')
df_impute=final_0.iloc[:,1:-1].fillna(0)
df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)

# Define function to flatten columns into a single row for each group
def flatten_columns(group):
    flattened = {}
    for col in group.columns:
        # Create a new column for each unique entry in the original column
        values = group[col].values
        for i, value in enumerate(values):
            new_col_name = f"{col}_{i+1}"
            flattened[new_col_name] = value
    return pd.Series(flattened)

# Apply the function to each group after grouping by 'yq'
result_df = df_raw.groupby('yq').apply(flatten_columns).reset_index()

# Display the resulting DataFrame
result_df

,yq,level_1,0
0,0.0,ym_1,198001.00
1,0.0,ym_2,198002.00
2,0.0,ym_3,198003.00
3,0.0,ym_4,198004.00
4,0.0,ym_5,198005.00
...,...,...,...
10999,200202.0,vol_1,0.00
11000,200202.0,shrout_1,0.00
11001,200202.0,medest_1,0.00
11002,200202.0,meanest_1,0.00


In [38]:
#check the value is observed in which month of a quarter
# Use 'thresh' parameter in dropna() to keep rows with at least 2 non-null values in the specified columns
filtered_df = result_df.dropna(subset=['value_1', 'value_2', 'value_3'], thresh=1)
# Display the filtered DataFrame
print("\nRows with more than 1 non-missing value:")
print(filtered_df[['value_1','value_2','value_3']])


Rows with more than 1 non-missing value:
    value_1  value_2  value_3
1       NaN      NaN  -0.0400
45      NaN      NaN  -0.0833
46      NaN      NaN   0.1472
48      NaN      NaN   0.2236
49      NaN      NaN  -0.0545
50      NaN      NaN   0.1309
51      NaN      NaN  -0.0600
52      NaN      NaN   0.1254
53      NaN      NaN  -0.0055
54      NaN      NaN   0.0763
55      NaN      NaN   0.0109
56      NaN      NaN  -0.1690
57      NaN      NaN  -0.0872
58      NaN      NaN  -0.1854
59      NaN      NaN  -0.1254
60      NaN      NaN  -0.1690
61      NaN      NaN  -0.0185
62      NaN      NaN   0.0463
63      NaN      NaN  -0.5370
64      NaN      NaN  -0.3148
65      NaN      NaN  -0.0648
66      NaN      NaN   0.0741
67      NaN      NaN  -0.4815
68      NaN      NaN  -0.1481
69      NaN      NaN  -0.2500
70      NaN      NaN  -0.0400
71      NaN      NaN  -0.2000
72      NaN      NaN  -0.0300
73      NaN      NaN  -0.0400
74      NaN      NaN  -0.1100
89      NaN      NaN  -0.030

,yq,ym,atq,ni,dv,acc,invest,mc,bm,dinvt,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,198001,198001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,198001,198002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,198001,198003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,198002,198004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,198002,198005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099281,200201,200202,41.0,-4.574,NaN,-0.456,-7.092,1.237606,NaN,0.074433,...,0.391155,0.117198,0.128440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1107318,200201,200203,41.0,-4.574,NaN,-0.456,-7.092,1.237606,NaN,0.074433,...,0.391155,0.117198,0.128440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1111738,200202,200204,42.9,-0.228,0.0,1.326,1.900,1.419927,NaN,-0.203119,...,0.091038,-0.283581,-0.161451,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117372,200202,200205,42.9,-0.228,0.0,1.326,1.900,1.419927,NaN,-0.203119,...,0.091038,-0.283581,-0.161451,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
tickers=df_EPS['ticker'].unique()
select_columns=['yq','ym','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

#df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})
df_0=df_EPS[df_EPS['ticker']==tickers[0]][select_columns]
#final_0 = pd.merge(df_date, df_0, how='left', on='date')
#df_impute=final_0.iloc[:,1:-1].fillna(0)
#df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)


In [12]:
df_EPS.dtypes

ticker       object
pends        object
ym            int64
permno      float64
gvkey       float64
datadate     object
yq          float64
cusip        object
value       float64
anndats      object
begin         int64
end           int64
seq           int64
fyearq      float64
fqtr        float64
fyr         float64
sic         float64
industry    float64
atq         float64
ni          float64
dv          float64
acc         float64
invest      float64
mc          float64
bm          float64
dinvt       float64
dar         float64
capx        float64
gm          float64
sga         float64
prc         float64
ret         float64
vol         float64
shrout      float64
medest      float64
meanest     float64
date         object
dtype: object

In [10]:
df_0

,yq,ym,atq,ni,dv,acc,invest,mc,bm,dinvt,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,NaN,198001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,198002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,198003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,198004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,198005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099281,NaN,200202,41.0,-4.574,NaN,-0.456,-7.092,1.237606,NaN,0.074433,...,0.391155,0.117198,0.128440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1107318,NaN,200203,41.0,-4.574,NaN,-0.456,-7.092,1.237606,NaN,0.074433,...,0.391155,0.117198,0.128440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1111738,NaN,200204,42.9,-0.228,0.0,1.326,1.900,1.419927,NaN,-0.203119,...,0.091038,-0.283581,-0.161451,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117372,NaN,200205,42.9,-0.228,0.0,1.326,1.900,1.419927,NaN,-0.203119,...,0.091038,-0.283581,-0.161451,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_EPS = pd.read_csv(os.path.join(args.root_path,args.data_path))
df_EPS

,ticker,pends,ym,permno,gvkey,datadate,yq,cusip,value,anndats,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,date
0,DANS,NaN,198001,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01
1,DANS,NaN,198002,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-02-01
2,DANS,NaN,198003,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-03-01
3,DANS,NaN,198004,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-04-01
4,DANS,NaN,198005,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236220,023A,2023-07-31,202307,17685.0,33301.0,2023-04-30,202303.0,25616310,0.72,2023-09-07,...,0.009274,-0.003308,-0.044046,51.090000,-0.094149,1313675.0,202371.0,0.66,0.66,2023-07-01
2236221,ANF,2023-07-31,202307,83976.0,63643.0,2023-04-30,202303.0,00289620,1.10,2023-08-23,...,-0.043901,-0.066070,0.113301,37.680000,0.214309,350453.0,50065.0,0.14,0.13,2023-07-01
2236222,ORCL,2023-08-31,202308,10104.0,12142.0,2023-05-31,202303.0,68389X10,1.19,2023-09-11,...,0.089011,-0.007677,-0.067191,117.230003,-0.012260,1563054.0,2714259.0,1.14,1.14,2023-08-01
2236223,ADBE,2023-08-31,202308,75510.0,12540.0,2023-05-31,202303.0,00724F10,4.09,2023-09-14,...,-0.292729,-0.004173,0.019000,546.169983,0.116935,607578.0,455800.0,3.88,3.89,2023-08-01


In [31]:
select_firm=[]
tickers=df_EPS['ticker'].unique()

for i in range(len(tickers)):
    select_columns=['date','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

    df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})
    df_0=df_EPS[df_EPS['ticker']==tickers[i]][select_columns]
    final_0 = pd.merge(df_date, df_0, how='left', on='date')
    df_impute=final_0.iloc[:,1:-1].fillna(0)
    df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)

    num_observed_vali=(~np.isnan(df_raw['value'][347:401])).sum()
    num_observed_test=(~np.isnan(df_raw['value'][401:])).sum()
    if num_observed_vali>=5 and num_observed_test>=5:
       select_firm.append(i)
       print(i,num_observed_vali,num_observed_test)

7 18 41
13 10 33
21 18 41
22 15 34
23 18 41
25 13 34
26 18 40
34 18 41
38 18 41
40 18 30
58 18 23
60 12 39
64 18 41
66 17 29
79 18 31
83 18 41
88 18 41
89 18 40
90 18 41
96 18 41
102 18 8
117 13 32
122 15 36
125 18 40
126 18 33
134 18 41
141 18 41
143 18 10
148 18 41
150 18 41
151 8 7
153 18 41
156 18 5
165 18 41
166 18 40
167 18 13
174 18 8
176 18 21
179 18 40
184 18 40
193 18 39
194 18 40
200 18 34
203 18 40
206 18 41
208 7 20
209 10 30
210 18 40
220 18 40
221 18 40
223 18 40
224 18 25
225 18 41
226 18 41
227 18 22
228 18 40
229 18 40
236 18 39
237 18 41
238 18 40
241 18 11
242 18 23
250 18 41
251 18 12
253 18 14
258 10 37
275 18 41
276 18 40
278 18 41
279 18 40
284 18 41
285 18 33
288 18 41
293 18 41
296 18 41
302 18 41
306 18 41
310 18 41
312 18 41
313 18 41
320 18 41
323 18 41
331 18 41
333 18 41
337 18 26
338 18 41
340 18 23
342 18 41
343 18 41
345 18 9
347 18 41
349 18 41
350 18 41
355 18 40
360 18 41
363 18 41
366 18 41
370 18 20
373 18 41
374 18 38
375 18 41
379 18 41
380 18 1

KeyboardInterrupt: 

In [44]:
i=10
df_select=df_EPS[df_EPS['ticker']==tickers[i]][select_columns]
df_select['value'].values

array([  nan,   nan,   nan,   nan,   nan,   nan,  2.12,   nan,   nan,
         nan,   nan,   nan,  2.16,   nan,   nan,  2.64,   nan,   nan,
        2.68,   nan,   nan,  3.72,   nan,   nan,  3.94,   nan,   nan,
        3.04,   nan,   nan,  3.72,   nan,   nan,  4.02,   nan,   nan,
        3.3 ,   nan,   nan, -0.52,   nan,   nan,   nan,   nan,   nan,
        3.6 ,   nan,   nan,  2.76,   nan,   nan,  2.36,   nan,   nan,
        2.08,   nan,   nan,  2.42,   nan,   nan,  2.76,   nan,   nan,
        3.22,   nan,   nan,  2.84,   nan,   nan,  3.64,   nan,   nan,
        7.76,   nan,   nan,  6.12,   nan,   nan, -3.48,   nan,   nan,
        3.44,   nan,   nan,  9.76,   nan,   nan,  4.32,   nan,   nan,
       -1.2 ,   nan,   nan,  4.6 ,   nan,   nan,  3.72,   nan,   nan,
        2.32,   nan,   nan,  5.36])

In [34]:
select_firm=[]
tickers=df_EPS['ticker'].unique()

for i in range(len(tickers)):
    select_columns=['date','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']

    df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})
    df_0=df_EPS[df_EPS['ticker']==tickers[i]][select_columns]
    final_0 = pd.merge(df_date, df_0, how='left', on='date')
    df_impute=final_0.iloc[:,1:-1].fillna(0)
    df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)

    num_observed_vali=(~np.isnan(df_raw['value'][347:401])).sum()
    num_observed_test=(~np.isnan(df_raw['value'][401:])).sum()
    if num_observed_vali>=5 and num_observed_test>=5:
       select_firm.append(i)
       print(i,num_observed_vali,num_observed_test)

7 18 41
13 10 33
21 18 41
22 15 34
23 18 41
25 13 34
26 18 40
34 18 41
38 18 41
40 18 30
58 18 23
60 12 39
64 18 41
66 17 29
79 18 31
83 18 41
88 18 41
89 18 40
90 18 41
96 18 41
102 18 8
117 13 32
122 15 36
125 18 40
126 18 33
134 18 41
141 18 41
143 18 10
148 18 41
150 18 41
151 8 7
153 18 41
156 18 5
165 18 41
166 18 40
167 18 13
174 18 8
176 18 21
179 18 40
184 18 40
193 18 39
194 18 40
200 18 34
203 18 40
206 18 41
208 7 20
209 10 30
210 18 40
220 18 40
221 18 40
223 18 40
224 18 25
225 18 41
226 18 41
227 18 22
228 18 40
229 18 40
236 18 39
237 18 41
238 18 40
241 18 11
242 18 23
250 18 41
251 18 12
253 18 14
258 10 37
275 18 41
276 18 40
278 18 41
279 18 40
284 18 41
285 18 33
288 18 41
293 18 41
296 18 41
302 18 41
306 18 41
310 18 41
312 18 41
313 18 41
320 18 41
323 18 41
331 18 41
333 18 41
337 18 26
338 18 41
340 18 23
342 18 41
343 18 41
345 18 9
347 18 41


KeyboardInterrupt: 

In [36]:
print(select_firm)

[7, 13, 21, 22, 23, 25, 26, 34, 38, 40, 58, 60, 64, 66, 79, 83, 88, 89, 90, 96, 102, 117, 122, 125, 126, 134, 141, 143, 148, 150, 151, 153, 156, 165, 166, 167, 174, 176, 179, 184, 193, 194, 200, 203, 206, 208, 209, 210, 220, 221, 223, 224, 225, 226, 227, 228, 229, 236, 237, 238, 241, 242, 250, 251, 253, 258, 275, 276, 278, 279, 284, 285, 288, 293, 296, 302, 306, 310, 312, 313, 320, 323, 331, 333, 337, 338, 340, 342, 343, 345, 347]


In [37]:
output_str = ' '.join(map(str, select_firm))

# Print the formatted string
print(output_str)

7 13 21 22 23 25 26 34 38 40 58 60 64 66 79 83 88 89 90 96 102 117 122 125 126 134 141 143 148 150 151 153 156 165 166 167 174 176 179 184 193 194 200 203 206 208 209 210 220 221 223 224 225 226 227 228 229 236 237 238 241 242 250 251 253 258 275 276 278 279 284 285 288 293 296 302 306 310 312 313 320 323 331 333 337 338 340 342 343 345 347


In [5]:
mses = []
maes = []

for ii in range(args.itr):

    setting = '{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_gl{}_df{}_eb{}_itr{}'.format(args.model_id, 18, args.label_len, args.pred_len,
                                                                    args.d_model, args.n_heads, args.e_layers, args.gpt_layers, 
                                                                    args.d_ff, args.embed, ii)
    path = os.path.join(args.checkpoints, setting)
    if not os.path.exists(path):
        os.makedirs(path)

    if args.freq == 0:
        args.freq = "M"
   

    train_data, train_loader = data_provider(args, 'train')
    vali_data, vali_loader = data_provider(args, 'val')
    test_data, test_loader = data_provider(args, 'test')
    
    

['date', 'atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm', 'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout', 'medest', 'meanest', 'value']
train 337
['date', 'atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm', 'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout', 'medest', 'meanest', 'value']
val 44
['date', 'atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm', 'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout', 'medest', 'meanest', 'value']
test 94


In [13]:
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(vali_loader)):
   # if (~batch_y.isnan()).sum()!=0:
       print(i)
       print('y',batch_y.shape)
       print(batch_y)
    

6it [00:00, 38.45it/s]

0
y torch.Size([1, 20, 1])
tensor([[[-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698]]], dtype=torch.float64)
1
y torch.Size([1, 20, 1])
tensor([[[-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698]]], dtype=torch.float64)
2
y torch.Size([1, 20, 1])
tensor([[[-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.

34it [00:00, 107.52it/s]

tensor([[[-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698]]], dtype=torch.float64)
25
y torch.Size([1, 20, 1])
tensor([[[-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698]]], dtype=torch.float64)
26
y torch.Size([1, 20, 1])
tensor([[[-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698],
         [-0.2698]

44it [00:00, 101.04it/s]


In [43]:
train_data[1][1]

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])

In [31]:
(~batch_y.isnan()).sum()

tensor(0)

In [32]:
batch_y.shape

torch.Size([1, 20, 1])

In [33]:
batch_y

tensor([[[nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan]]], dtype=torch.float64)

In [35]:
data_x

NameError: name 'data_x' is not defined

In [114]:
batch_x.shape

torch.Size([512, 19, 1])

In [15]:
batch_x[0,:,:]

tensor([[-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698],
        [-0.2698]], dtype=torch.float64)

In [6]:
    device = torch.device('cuda:0')

    time_now = time.time()
    train_steps = len(train_loader)

    if args.model == 'PatchTST':
        model = PatchTST(args, device)
        model.to(device)
    elif args.model == 'DLinear':
        model = DLinear(args, device)
        model.to(device)
    else:
        model = GPT4TS(args, device)
    # mse, mae = test(model, test_data, test_loader, args, device, ii)

    params = model.parameters()
    model_optim = torch.optim.Adam(params, lr=args.learning_rate)
    
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    if args.loss_func == 'mse':
        criterion = nn.MSELoss()
    elif args.loss_func == 'smape':
        class SMAPE(nn.Module):
            def __init__(self):
                super(SMAPE, self).__init__()
            def forward(self, pred, true):
                return torch.mean(200 * torch.abs(pred - true) / (torch.abs(pred) + torch.abs(true) + 1e-8))
        criterion = SMAPE()
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=args.tmax, eta_min=1e-8)

    for epoch in range():

        iter_count = 0
        train_loss = []
        epoch_time = time.time()
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(train_loader)):

            iter_count += 1
            model_optim.zero_grad()
            batch_x = batch_x.float().to(device)

            batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)
            
            outputs = model(batch_x, ii)
 
            outputs = outputs[:, -args.pred_len:, :]
            batch_y = batch_y[:, -args.pred_len:, :].to(device)

            missing_idx=~torch.isnan(batch_y)
            outputs=outputs[missing_idx]
            batch_y=batch_y[missing_idx]
            
            loss = criterion(outputs, batch_y)
            print(loss)
            train_loss.append(loss.item())
            

gpt2 = GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, ele

6it [00:00,  8.31it/s]

tensor(2.0324, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.6160, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.9667, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.2905, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.0565, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.1297, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.3761, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.0249, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.0587, device='cuda:0', grad_fn=<MseLossBackward0>)


11it [00:01, 10.02it/s]

tensor(1.9003, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.3872, device='cuda:0', grad_fn=<MseLossBackward0>)


...


5it [00:00, 41.20it/s]

tensor(2.1837, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.2256, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.6028, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.1747, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.2088, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.9953, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.3395, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2.0917, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.9044, device='cuda:0', grad_fn=<MseLossBackward0>)


10it [00:00, 38.37it/s]


tensor(2.0890, device='cuda:0', grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 

In [9]:
batch_x.shape

torch.Size([138, 18, 1])

In [10]:
batch_y.shape

torch.Size([138, 21, 1])

In [40]:
class Dataset_EPS(Dataset):
    def __init__(self, root_path, flag='train', size=None,
                 features='S', data_path='ETTh1.csv',
                 target='OT', scale=True, timeenc=0, freq='h',
                 percent=10, max_len=-1, train_all=False,idx=0):
        # size [seq_len, label_len, pred_len]
        # info
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq
        self.percent = percent
        self.idx=idx

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()
        self.enc_in = self.data_x.shape[-1]
        self.tot_len = len(self.data_x) - self.seq_len - self.pred_len + 1
        

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_EPS = pd.read_csv(os.path.join(self.root_path,
                                          self.data_path))
        
        select_columns=['date','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']
        tickers=df_EPS['ticker'].unique()
        df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})

        df_0=df_EPS[df_EPS['ticker']==tickers[self.idx]][select_columns]
        final_0 = pd.merge(df_date, df_0, how='left', on='date')
        df_impute=final_0.iloc[:,1:-1].fillna(0)
        df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)
       
        

        '''
        df_raw.columns: ['date', ...(other features), target feature]
        '''
        cols = list(df_raw.columns)
        print(cols)
        cols.remove(self.target)
        cols.remove('date')
        df_raw = df_raw[['date'] + cols + [self.target]]
        # print(cols)
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]
        
        if self.set_type == 0:
            border2 = (border2 - self.seq_len) * self.percent // 100 + self.seq_len

        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        df_stamp = df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)

        self.data_x = data[:,:-1][border1:border2]
        self.data_y = data[:,-1:][border1:border2]
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        feat_id = index // self.tot_len
        s_begin = index % self.tot_len
        
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len
        seq_x = self.data_x[s_begin:s_end, feat_id:feat_id+1]
        #seq_y = self.data_y[r_begin:r_end, feat_id:feat_id+1]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return (len(self.data_x) - self.seq_len - self.pred_len + 1) * self.enc_in

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

NameError: name 'Dataset' is not defined

In [5]:
 df_EPS = pd.read_csv(os.path.join(args.root_path,args.data_path))

In [6]:
df_EPS

,ticker,pends,ym,permno,gvkey,datadate,yq,cusip,value,anndats,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,date
0,DANS,NaN,198001,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01
1,DANS,NaN,198002,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-02-01
2,DANS,NaN,198003,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-03-01
3,DANS,NaN,198004,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-04-01
4,DANS,NaN,198005,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236220,023A,2023-07-31,202307,17685.0,33301.0,2023-04-30,202303.0,25616310,0.72,2023-09-07,...,0.009274,-0.003308,-0.044046,51.090000,-0.094149,1313675.0,202371.0,0.66,0.66,2023-07-01
2236221,ANF,2023-07-31,202307,83976.0,63643.0,2023-04-30,202303.0,00289620,1.10,2023-08-23,...,-0.043901,-0.066070,0.113301,37.680000,0.214309,350453.0,50065.0,0.14,0.13,2023-07-01
2236222,ORCL,2023-08-31,202308,10104.0,12142.0,2023-05-31,202303.0,68389X10,1.19,2023-09-11,...,0.089011,-0.007677,-0.067191,117.230003,-0.012260,1563054.0,2714259.0,1.14,1.14,2023-08-01
2236223,ADBE,2023-08-31,202308,75510.0,12540.0,2023-05-31,202303.0,00724F10,4.09,2023-09-14,...,-0.292729,-0.004173,0.019000,546.169983,0.116935,607578.0,455800.0,3.88,3.89,2023-08-01


In [79]:
df_EPS['ym'].nunique()

524

In [8]:
select_columns=['date','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']
tickers=df_EPS['ticker'].unique()
df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})

df_0=df_EPS[df_EPS['ticker']==tickers[0]][select_columns]
final_0 = pd.merge(df_date, df_0, how='left', on='date')
df_impute=final_0.iloc[:,1:-1].fillna(0)
df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)
(~np.isnan(df_raw['value'][347:401])).sum()
       

0

In [60]:
df_raw['value'].values

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan, -2.150e-02,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan, -4.280e-02,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,  2.200e-03,
              nan,        nan,  1.800e-03,        nan,        nan,
        3.100e-03,        nan,        nan,  1.800e-03,        nan,
              nan,  2.200e-03,        nan,        nan,  1.100e-03,
              nan,        nan,  2.200e-03,        nan,        

In [81]:
(df_raw['atq']==0).sum()

26

In [82]:
df_raw['value'].isnull().sum()

365

In [25]:
df_EPS

,ticker,pends,ym,permno,gvkey,datadate,yq,cusip,value,anndats,...,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,date
0,DANS,NaN,198001,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-01-01
1,DANS,NaN,198002,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-02-01
2,DANS,NaN,198003,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-03-01
3,DANS,NaN,198004,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-04-01
4,DANS,NaN,198005,77861.0,25669.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236220,023A,2023-07-31,202307,17685.0,33301.0,2023-04-30,202303.0,25616310,0.72,2023-09-07,...,0.009274,-0.003308,-0.044046,51.090000,-0.094149,1313675.0,202371.0,0.66,0.66,2023-07-01
2236221,ANF,2023-07-31,202307,83976.0,63643.0,2023-04-30,202303.0,00289620,1.10,2023-08-23,...,-0.043901,-0.066070,0.113301,37.680000,0.214309,350453.0,50065.0,0.14,0.13,2023-07-01
2236222,ORCL,2023-08-31,202308,10104.0,12142.0,2023-05-31,202303.0,68389X10,1.19,2023-09-11,...,0.089011,-0.007677,-0.067191,117.230003,-0.012260,1563054.0,2714259.0,1.14,1.14,2023-08-01
2236223,ADBE,2023-08-31,202308,75510.0,12540.0,2023-05-31,202303.0,00724F10,4.09,2023-09-14,...,-0.292729,-0.004173,0.019000,546.169983,0.116935,607578.0,455800.0,3.88,3.89,2023-08-01


In [51]:
df_raw

,date,atq,ni,dv,acc,invest,mc,bm,dinvt,dar,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,1980-01-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,NaN
1,1980-02-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,NaN
2,1980-03-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,NaN
3,1980-04-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,NaN
4,1980-05-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,2023-04-01,76445.0,3362.0,7789.0,-11253.0,-421.0,12.704897,-5.351174,0.045843,-0.032969,0.003714,0.019924,0.090754,295.119995,0.002259,1043572.0,1014956.0,3.85,3.85,3.82
520,2023-05-01,76386.0,3873.0,2118.0,-1741.0,-59.0,12.620312,-6.728668,-0.020309,0.230326,-0.052159,-0.013112,-0.069421,300.540009,0.018365,624846.0,1014956.0,0.00,0.00,NaN
521,2023-06-01,76386.0,3873.0,2118.0,-1741.0,-59.0,12.620312,-6.728668,-0.020309,0.230326,-0.052159,-0.013112,-0.069421,283.450012,-0.049910,1167558.0,1005376.0,0.00,0.00,NaN
522,2023-07-01,76386.0,3873.0,2118.0,-1741.0,-59.0,12.620312,-6.728668,-0.020309,0.230326,-0.052159,-0.013112,-0.069421,310.640015,0.095925,925631.0,1005376.0,4.47,4.44,4.65


In [14]:
        cols = list(df_raw.columns)
        print(cols)
        cols.remove(args.target)
        cols.remove('date')
        df_raw = df_raw[['date'] + cols + [args.target]]
        # print(cols)
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - args.seq_len, len(df_raw) - num_test - args.seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[1]
        border2 = border2s[1]
        
        
        if args.features == 'M' or args.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif args.features == 'S':
            df_data = df_raw[[args.target]]


        df_stamp = df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if 0 == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], 1).values
        data = df_data.values
        data_x = data[:,:-1][border1:border2]
        data_y = data[:,-1:][border1:border2]
        data_stamp = data_stamp

['date', 'atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm', 'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout', 'medest', 'meanest', 'value']


In [15]:
border1s

[0, 347, 401]

In [61]:
data_y

array([[ nan],
       [0.19],
       [ nan],
       [ nan],
       [0.35],
       [ nan],
       [ nan],
       [0.64],
       [ nan],
       [ nan],
       [0.41],
       [ nan],
       [ nan],
       [0.24],
       [ nan],
       [ nan],
       [0.45],
       [ nan],
       [ nan],
       [0.72],
       [ nan],
       [ nan],
       [0.51],
       [ nan],
       [ nan],
       [0.36],
       [ nan],
       [ nan],
       [0.5 ],
       [ nan],
       [ nan],
       [0.86],
       [ nan],
       [ nan],
       [0.6 ],
       [ nan],
       [ nan],
       [0.5 ],
       [ nan],
       [ nan],
       [0.65],
       [ nan],
       [ nan],
       [1.01],
       [ nan],
       [ nan],
       [0.74],
       [ nan],
       [ nan],
       [0.67],
       [ nan],
       [ nan],
       [0.83],
       [ nan],
       [ nan],
       [1.24],
       [ nan],
       [ nan],
       [0.95],
       [ nan],
       [ nan],
       [0.73],
       [ nan],
       [ nan],
       [0.96],
       [ nan],
       [ n

In [45]:
enc_in = data_x.shape[-1]
tot_len = len(data_x) - args.seq_len - args.pred_len + 1
print(enc_in,tot_len)
      

18 337


In [57]:
        index=7000
        feat_id = index // tot_len
        s_begin = index % tot_len
        
        s_end = s_begin + args.seq_len
        r_begin = s_end - args.label_len
        r_end = r_begin + args.label_len + args.pred_len
        seq_x = data_x[s_begin:s_end, :]
        #seq_y = self.data_y[r_begin:r_end, feat_id:feat_id+1]
        seq_y = data_y[r_begin:r_end]
        seq_x_mark = data_stamp[s_begin:s_end]
        seq_y_mark = data_stamp[r_begin:r_end]

In [56]:
s_end

301

In [58]:
seq_y

array([[0.5 ],
       [ nan],
       [ nan],
       [0.4 ],
       [ nan],
       [ nan],
       [0.3 ],
       [ nan],
       [ nan],
       [0.39],
       [ nan],
       [ nan],
       [0.56],
       [ nan],
       [ nan],
       [0.5 ],
       [ nan],
       [ nan],
       [0.42],
       [ nan]])

In [103]:
seq_y.shape

(20, 1)

In [96]:
seq_y

array([[    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [-0.0215],
       [    nan]])

In [89]:
seq_y.shape

(20, 1)

In [75]:
data_x

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [4.087700e+04, 3.010000e+02, 1.525000e+03, ..., 1.693342e+06,
        3.800000e-01, 3.800000e-01],
       [4.361900e+04, 7.250000e+02, 3.990000e+02, ..., 1.693342e+06,
        0.000000e+00, 0.000000e+00],
       [4.361900e+04, 7.250000e+02, 3.990000e+02, ..., 1.693342e+06,
        0.000000e+00, 0.000000e+00]])

In [6]:
 for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(train_loader)):
     print(batch_x.shape)
     print(batch_y.shape)


12it [00:00, 105.67it/s]

torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
torch.Size([512, 21, 1])
torch.Size([434, 18, 1])
torch.Size([434, 21, 1])


In [8]:
 for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(test_loader)):
     print(batch_x.shape)
     print(batch_x)
     print(batch_y.shape)

4it [00:00, 81.30it/s]

torch.Size([512, 18, 1])
tensor([[[-0.6957],
         [-0.6957],
         [-0.6957],
         ...,
         [-0.6957],
         [-0.6957],
         [-0.6957]],

        [[-0.6957],
         [-0.6957],
         [-0.6957],
         ...,
         [-0.6957],
         [-0.6957],
         [-0.6957]],

        [[-0.6957],
         [-0.6957],
         [-0.6957],
         ...,
         [-0.6957],
         [-0.6957],
         [-0.6957]],

        ...,

        [[-0.6535],
         [-0.6535],
         [-0.6535],
         ...,
         [-0.6535],
         [-0.6535],
         [-0.6535]],

        [[-0.6535],
         [-0.6535],
         [-0.6535],
         ...,
         [-0.6535],
         [-0.6535],
         [-0.6535]],

        [[-0.6535],
         [-0.6535],
         [-0.6535],
         ...,
         [-0.6535],
         [-0.6535],
         [-0.6535]]], dtype=torch.float64)
torch.Size([512, 21, 1])
torch.Size([512, 18, 1])
tensor([[[-0.6535],
         [-0.6535],
         [-0.6535],
         ...,


In [7]:
outputs.shape

torch.Size([512, 12, 1])

In [10]:
batch_y.shape

torch.Size([512, 21])

In [7]:
batch_x

NameError: name 'batch_x' is not defined

In [7]:
        import pandas as pd
        df_EPS = pd.read_csv(os.path.join(args.root_path,
                                          args.data_path))
        
        select_columns=['date','atq', 'ni', 'dv', 'acc', 'invest', 'mc', 'bm',
                     'dinvt', 'dar', 'capx', 'gm', 'sga', 'prc', 'ret', 'vol', 'shrout',
                     'medest', 'meanest','value']
        tickers=df_EPS['ticker'].unique()
        df_date=pd.DataFrame({'date':df_EPS['date'].dropna().unique()})

        df_0=df_EPS[df_EPS['ticker']==tickers[0]][select_columns]
        final_0 = pd.merge(df_date, df_0, how='left', on='date')
        df_impute=final_0.iloc[:,1:-1].fillna(0)
        df_raw = pd.concat([final_0.iloc[:, :1], df_impute, final_0.iloc[:, -1:]], axis=1)
         

In [10]:
df_EPS.describe()

,ym,permno,gvkey,yq,value,begin,end,seq,fyearq,fqtr,...,dar,capx,gm,sga,prc,ret,vol,shrout,medest,meanest
count,2.236225e+06,2.235274e+06,2.234767e+06,713174.000000,684407.000000,2.236225e+06,2.236225e+06,2.236225e+06,2.192567e+06,2.192272e+06,...,1.756122e+06,1.728486e+06,1.888651e+06,1.506321e+06,2.059437e+06,2.050243e+06,2.059419e+06,2.079102e+06,532579.00000,532579.000000
mean,2.002556e+05,6.157964e+04,4.640005e+04,200285.762015,0.168323,1.991954e+05,2.013041e+05,1.270881e+02,2.002446e+03,2.497810e+00,...,3.993924e-01,-6.321864e-01,9.485635e-02,-1.569992e-01,2.590775e+01,1.253586e-02,1.495523e+05,8.819640e+04,0.27280,0.273052
std,1.048453e+03,2.848655e+04,5.765498e+04,1038.395867,2.280542,9.310580e+02,1.002140e+03,1.059575e+02,1.043082e+01,1.119223e+00,...,3.151203e+02,7.350002e+01,6.307295e+01,4.269853e+01,4.694364e+01,1.644536e-01,7.975488e+05,3.475159e+05,1.34245,1.338396
min,1.980010e+05,1.000100e+04,1.004000e+03,198002.000000,-434.783000,1.979120e+05,1.985030e+05,1.000000e+00,1.980000e+03,1.000000e+00,...,-3.139937e+04,-3.722266e+04,-1.254369e+04,-2.513782e+04,2.080000e-02,-9.526842e-01,0.000000e+00,2.200000e+01,-217.20000,-226.130000
25%,1.994110e+05,3.555300e+04,8.823000e+03,199501.000000,0.006300,1.983110e+05,2.005060e+05,4.500000e+01,1.994000e+03,1.000000e+00,...,-9.966298e-02,-1.845640e-01,-6.436100e-02,-9.057730e-02,7.310000e+00,-6.153846e-02,3.940000e+03,8.659000e+03,0.03000,0.040000
50%,2.002050e+05,7.652900e+04,1.977100e+04,200204.000000,0.150000,1.990090e+05,2.015090e+05,9.700000e+01,2.002000e+03,2.000000e+00,...,0.000000e+00,1.557987e-02,0.000000e+00,-4.138952e-03,1.650000e+01,4.221936e-03,1.943300e+04,2.260400e+04,0.19000,0.190000
75%,2.011040e+05,8.453600e+04,6.328100e+04,201103.000000,0.400000,1.998030e+05,2.023060e+05,1.830000e+02,2.011000e+03,3.000000e+00,...,1.024314e-01,2.000487e-01,6.171264e-02,8.286999e-02,3.105000e+01,7.299270e-02,8.649450e+04,5.999500e+04,0.44000,0.440000
max,2.023080e+05,9.343600e+04,3.470070e+05,202303.000000,295.600000,2.020090e+05,2.023080e+05,5.000000e+02,2.023000e+03,4.000000e+00,...,2.173947e+05,1.279401e+02,3.139100e+04,9.406193e+02,3.507070e+03,1.625053e+01,2.012427e+08,2.920640e+07,105.35000,87.680000


In [8]:
df_raw

,date,atq,ni,dv,acc,invest,mc,bm,dinvt,dar,capx,gm,sga,prc,ret,vol,shrout,medest,meanest,value
0,1980-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1980-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1980-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1980-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1980-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,2023-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
520,2023-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
521,2023-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
522,2023-07-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [22]:
data = df_data.values

In [25]:
data[:,:-1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
data[:,-1:]

array([[    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [-0.04  ],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [    nan],
       [  

In [7]:
    device = torch.device('cuda:0')

    time_now = time.time()
    train_steps = len(train_loader)

    if args.model == 'PatchTST':
        model = PatchTST(args, device)
        model.to(device)
    elif args.model == 'DLinear':
        model = DLinear(args, device)
        model.to(device)
    else:
        model = GPT4TS(args, device)
    # mse, mae = test(model, test_data, test_loader, args, device, ii)

    params = model.parameters()
    model_optim = torch.optim.Adam(params, lr=args.learning_rate)
    
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    if args.loss_func == 'mse':
        criterion = nn.MSELoss()
    elif args.loss_func == 'smape':
        class SMAPE(nn.Module):
            def __init__(self):
                super(SMAPE, self).__init__()
            def forward(self, pred, true):
                return torch.mean(200 * torch.abs(pred - true) / (torch.abs(pred) + torch.abs(true) + 1e-8))
        criterion = SMAPE()
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=args.tmax, eta_min=1e-8)

    for epoch in range(args.train_epochs):

        iter_count = 0
        train_loss = []
        epoch_time = time.time()
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(train_loader)):

            iter_count += 1
            model_optim.zero_grad()
            batch_x = batch_x.float().to(device)

            batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)
            
            outputs = model(batch_x, ii)

            outputs = outputs[:, -args.pred_len:, :]
            batch_y = batch_y[:, -args.pred_len:, :].to(device)
            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

            if (i + 1) % 1000 == 0:
                print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                speed = (time.time() - time_now) / iter_count
                left_time = speed * ((args.train_epochs - epoch) * train_steps - i)
                print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                iter_count = 0
                time_now = time.time()
            loss.backward()
            model_optim.step()

        
        print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))

        train_loss = np.average(train_loss)
        vali_loss = vali(model, vali_data, vali_loader, criterion, args, device, ii)
        # test_loss = vali(model, test_data, test_loader, criterion, args, device, ii)
        # print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f}, Test Loss: {4:.7f}".format(
        #     epoch + 1, train_steps, train_loss, vali_loss, test_loss))
        print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f}".format(
            epoch + 1, train_steps, train_loss, vali_loss))

        if args.cos:
            scheduler.step()
            print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
        else:
            adjust_learning_rate(model_optim, epoch + 1, args)
        early_stopping(vali_loss, model, path)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    best_model_path = path + '/' + 'checkpoint.pth'
    model.load_state_dict(torch.load(best_model_path))
    print("------------------------------------")
    mse, r2 = test(model, test_data, test_loader, args, device, ii)
    mses.append(mse)
    maes.append(r2)

mses = np.array(mses)
maes = np.array(maes)
print("mse_mean = {:.4f}, mse_std = {:.4f}".format(np.mean(mses), np.std(mses)))
print("r2_mean = {:.4f}, mae_std = {:.4f}".format(np.mean(r2), np.std(r2)))

gpt2 = GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, ele

12it [00:01,  8.62it/s]

Epoch: 1 cost time: 1.6589577198028564



1it [00:00, 10.12it/s]


Epoch: 1, Steps: 12 | Train Loss: nan Vali Loss: nan
lr_adjust = {1: 0.0001}
Updating learning rate to 0.0001
Validation loss decreased (inf --> nan).  Saving model ...


...

Epoch: 10 cost time: 0.8859546184539795


1it [00:00, 11.69it/s]


Epoch: 10, Steps: 12 | Train Loss: nan Vali Loss: nan
lr_adjust = {10: 4.782969000000001e-05}
Updating learning rate to 4.782969000000001e-05
Validation loss decreased (nan --> nan).  Saving model ...
------------------------------------


Exception ignored in: <function _releaseLock at 0x7fcf867245e0>
Traceback (most recent call last):
  File "/home/dan/anaconda3/envs/geo/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
3it [00:00, 25.94it/s]

test shape: (3, 6144) (3, 6144)
test shape: (1, 3, 6144) (1, 3, 6144)
mae:nan, mse:nan, rmse:nan, r2:nan
mse_mean = nan, mse_std = nan
r2_mean = nan, mae_std = nan


train 1121152
val 148903
test 324083


In [93]:
len(train_data)//8759

128

In [94]:
len(vali_data)/8759

17.0

In [95]:
len(test_data)/8759

37.0

In [96]:
128+17+37

182

In [68]:
train_data[0][0].shape

(12, 1)

In [69]:
train_data[0][1].shape

(15, 1)

In [70]:
train_data[0][2].shape

(12, 1)

In [71]:
train_data[0][3].shape

(15, 1)

In [77]:
train_data[0][3]

array([[ 0.31818182],
       [ 0.40909091],
       [ 0.5       ],
       [-0.5       ],
       [-0.40909091],
       [-0.31818182],
       [-0.22727273],
       [-0.13636364],
       [-0.04545455],
       [ 0.04545455],
       [ 0.13636364],
       [ 0.22727273],
       [ 0.31818182],
       [ 0.40909091],
       [ 0.5       ]])

In [78]:
78831/8759

9.0